Converts the output of the cnn into useable k-fold dataframes

In [1]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import paths
import shutil
from PIL import Image
import math
from os.path import join
from os import listdir
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
k_ids = ['kfold_0_ids.npy', 'kfold_1_ids.npy', 'kfold_2_ids.npy', 
       'kfold_3_ids.npy', 'kfold_4_ids.npy']
k_preds = ['kfold_0_preds.npy', 'kfold_1_preds.npy', 'kfold_2_preds.npy', 
       'kfold_3_preds.npy', 'kfold_4_preds.npy']
test_ids_path = 'cnn_test_filenames.npy'
test_preds_path = 'cnn_test_preds.npy'

In [3]:
train_json = pd.read_json(paths.TRAIN_JSON)
listing_ids = train_json.loc[:,'listing_id']
test_json = pd.read_json(paths.TEST_JSON)
test_listing_ids = test_json.loc[:,'listing_id']

In [33]:
# concat all of the k fold predictions
index =0
folds = []
fold_ids = []

# RANDOM SEED IS 2016. IDK WHY BUT IT IS @MICHAEL ZHAO
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for trn_indices, val_indices in kf.split(range(len(listing_ids))):
    
    # these are the ids we need
    val_ids = listing_ids.iloc[val_indices]
    fold_ids.append(val_ids)
    curr_ids = np.load(join(paths.CNN_OUTPUT_DIR, k_ids[index]))
    curr_preds = np.load(join(paths.CNN_OUTPUT_DIR, k_preds[index]))
    # slice the extra preds (happens due to generators smh)
    curr_preds = curr_preds[:len(curr_ids)]
    
    # create a new dataframe with the info
    X_trn = pd.DataFrame([curr_ids, curr_preds[:,0], curr_preds[:,1], 
                         curr_preds[:,2]])
    # label the columns
    X_trn.rename(index = {0:'listing_id', 
                           1:'low', 
                           2:'medium', 
                           3:'high'}, inplace=True)  
    X_trn = X_trn.T
    
    # convert names to listing_id
    X_trn['listing_id'] = X_trn['listing_id'].apply(lambda x: 
                                x[x.index('/')+1:x.index('_')])
    folds.append(X_trn)
    index += 1  

In [34]:
# do the same for test data
test_ids = np.load(join(paths.CNN_OUTPUT_DIR, test_ids_path))
test_preds = np.load(join(paths.CNN_OUTPUT_DIR, test_preds_path))
test_preds = test_preds[:len(test_ids)]
X_test = pd.DataFrame([test_ids, test_preds[:,0], test_preds[:,1], 
                         test_preds[:,2]])
X_test.rename(index = {0:'listing_id', 
                           1:'low', 
                           2:'medium', 
                           3:'high'}, inplace=True)  
X_test = X_test.T
X_test['listing_id'] = X_test['listing_id'].apply(lambda x: 
                                x[x.index('/')+1:x.index('_')])

In [35]:
# group by listing id and merge by mean
grouped_folds = []
for X_trn in folds:
    X_trn = X_trn.apply(pd.to_numeric)
    X_grouped = X_trn.groupby('listing_id').mean()
    grouped_folds.append(X_grouped)
X_test = X_test.apply(pd.to_numeric)
X_test_grouped = X_test.groupby('listing_id').mean()

In [37]:
# merge with listing_ids (fills NaN for listings w/o images i think)
final_folds = []
for X_grouped, val_ids in zip(grouped_folds, fold_ids):
    X_merged = pd.DataFrame(val_ids)
    X_merged = X_merged.join(X_grouped, on='listing_id', how='left')
    final_folds.append(X_merged)
X_test_merged = pd.DataFrame(test_listing_ids)
X_test_merged = X_test_merged.join(X_test_grouped, 
                                   on='listing_id', how='left')

In [47]:
for i in range(len(final_folds)):
    final_folds[i].to_csv(join(paths.CNN_OUTPUT_DIR,"kfold_{}.csv".format(i)))

In [40]:
X_test_merged.to_csv(join(paths.CNN_OUTPUT_DIR,"cnn_test.csv"))